# Lumber Transportation Problem 
J. Reeb and S. Leavengood

Millco has three wood mills and is planning three new logging sites. Each mill has a maximum capacity and each logging site can harvest a certain number of truckloads of lumber per day. The cost of a haul is $2/mile of distance. If distances from logging sites to mills are given below, how should the hauls be routed to minimize hauling costs while meeting all demands?

| Logging Site | Mill A | Mill B | Mill C | Max loads per day |
|:------------:|:------:|:------:|:------:|:-----------------:|
| 1            |  8     |  15    |  50    |  20               |
| 2            |  10    |  17    |  20    |  30               |
| 3            |  30    |  26    |  15    |  45               |
| Mill demand  |  30    |  35    |  30    |                   |

In [ ]:
# Install dependencies
! pip install -q gamspy

In [ ]:
from gamspy import Container, Set, Alias, Parameter, Variable, Equation, Model, Sum, Sense
import numpy as np

In [ ]:
m = Container()

# Model sets and parameters
sites = Set(container=m, name="sites", records=["1", "2", "3"])
mills = Set(container=m, name="mills", records=["Mill A", "Mill B", "Mill C"])
dist = Parameter(
    container=m,
    name="dist",
    domain=[sites, mills],
    records=np.array([[8, 15, 50], [10, 17, 20], [30, 26, 15]]),
)
supply = Parameter(
    container=m, name="supply", domain=sites, records=np.array([20, 30, 45])
)
demand = Parameter(
    container=m, name="demand", domain=mills, records=np.array([30, 35, 30])
)
cost_per_haul = 4

# Model variables
Alias(container=m, name="s", alias_with=sites)
Alias(container=m, name="m", alias_with=mills)
ship = Variable(container=m, name="ship", type="positive", domain=[sites, mills])

defcost = cost_per_haul * Sum([sites, mills], ship[sites, mills] * dist[sites, mills])

defsupply = Equation(container=m, name="defsupply", domain=sites)
defsupply[sites] = Sum(mills, ship[sites, mills]) == supply[sites]

defdemand = Equation(container=m, name="defdemand", domain=mills)
defdemand[mills] = Sum(sites, ship[sites, mills]) == demand[mills]

millco = Model(
    container=m,
    name="millco",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=defcost,
)

In [ ]:
millco.solve()

In [ ]:
ship.records.pivot(index='sites', columns='mills', values='level')

In [ ]:
print(f'Total cost will be {millco.objective_value}')

In [ ]:
m = Container()

# Generic Model sets and parameters
i = Set(
    container=m,
    name="i",
    records=["i" + str(i) for i in range(6)],
    description="equation index",
)
j = Set(
    container=m,
    name="j",
    records=["j" + str(j) for j in range(9)],
    description="variable index",
)
A = Parameter(
    container=m,
    name="A",
    domain=[i, j],
    records=np.array(
        [
            [1, 1, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 1, 1],
            [1, 0, 0, 1, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0, 0, 1, 0],
            [0, 0, 1, 0, 0, 1, 0, 0, 1],
        ]
    ),
)
b = Parameter(
    container=m, name="b", domain=i, records=np.array([20, 30, 45, 30, 35, 30])
)
c = Parameter(
    container=m,
    name="c",
    domain=j,
    records=np.array([8, 15, 50, 10, 17, 20, 30, 26, 15]),
)
x = Variable(container=m, name="x", type="positive", domain=j)

obj = 4 * Sum(j, c[j] * x[j])

e = Equation(container=m, name="e", domain=i)
e[i] = Sum(j, A[i, j] * x[j]) == b[i]

generic_model = Model(
    container=m,
    name="generic_model",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

In [ ]:
generic_model.solve()

In [ ]:
x.records

In [ ]:
print(f'Total cost will be {generic_model.objective_value}')